In [4]:
import numpy
import requests
import gzip
import os
import hashlib
import math

In [5]:
def Fetch(url, path):
    if not os.path.exists(path):
        os.makedirs(path)

    f_path = os.path.join(path, hashlib.md5(url.encode('utf-8')).hexdigest())

    if os.path.isfile(f_path):
        with open(f_path, "rb") as f:
            data = f.read()

    else:
        with open(f_path, "wb") as f:
            data = requests.get(url).content
            f.write(data)

    return numpy.frombuffer(gzip.decompress(data), dtype=numpy.uint8).copy()

def GetNumpy():
    X_train = Fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "../dataset/mnist/")[0x10:].reshape((-1, 28, 28))
    Y_train = Fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz", "../dataset/mnist/")[0x8:]
    X_test  = Fetch("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz", "../dataset/mnist/")[0x10:].reshape((-1, 28, 28))
    Y_test  = Fetch("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz", "../dataset/mnist/")[0x8:]
    return X_train, Y_train, X_test, Y_test

In [6]:
def Xavier(weights, nodes):
    arg = 1 / math.sqrt(weights * nodes)
    w = numpy.random.uniform(-arg, arg, (weights, nodes))
    b = numpy.zeros((1, nodes))
    return w, b

def GetFans(shape):
    fanIn = shape[0] if len(shape) == 2 else np.prod(shape[1:])
    fanOut = shape[1] if len(shape) == 2 else shape[0]
    return fanIn, fanOut

In [31]:
X_train, Y_train, X_test, Y_test = GetNumpy()
sample = X_train[0:32]

Y_batch = numpy.zeros((32, 10), numpy.float64)
Y_batch[range(Y_batch.shape[0]), Y_train[0:32]] = 1

ALPHA = 0.0025

### Convolutional layer
TODO: automate padding

In [32]:
kernelShape = (32, 3, 3)

fanIn, fanOut = GetFans(kernelShape[1:])
limit = numpy.sqrt(6. / (fanIn + fanOut))
kernel = numpy.random.uniform(-limit, limit, kernelShape)

padding = (1, 1)
stride = 1

if (padding):
    padded = numpy.zeros((sample.shape[0], sample.shape[1] + 2*padding[0], sample.shape[2] + 2*padding[1]))
    padded[:, padding[0]:sample.shape[1]+padding[0], padding[1]:sample.shape[2]+padding[1]] = sample
else:
    padded = sample

conv2d1 = numpy.zeros((
    sample.shape[0] * kernel.shape[0],
    int(((sample.shape[1] - kernel.shape[1] + 2 * padding[0]) / stride) + 1),
    int(((sample.shape[2] - kernel.shape[2] + 2 * padding[1]) / stride) + 1)
))

for i in range(padded.shape[0]):
    for x in range(conv2d1.shape[1]):
        for y in range(conv2d1.shape[2]):
            mul = padded[i, x:x+kernel.shape[1], y:y+kernel.shape[2]] * kernel
            conv2d1[i*kernel.shape[0]:(i+1)*kernel.shape[0], x, y] = numpy.sum(mul, axis=(1, 2))

In [33]:
act1 = numpy.maximum(0, conv2d1)

### Pooling
Average

In [34]:
pool1shape = (2, 2)

pool1 = numpy.zeros((act1.shape[0], int(act1.shape[1] / pool1shape[0]), int(act1.shape[2] / pool1shape[1])))

for x in range(pool1.shape[1]):
    for y in range(pool1.shape[2]):
        pool1[:, x, y] = numpy.max(act1[:,x*pool1shape[0]:(x+1)*pool1shape[0],y*pool1shape[1]:(y+1)*pool1shape[1]], axis=(1, 2))

In [35]:
kernelShape2 = (64, 3, 3)
fanIn2, fanOut2 = GetFans(kernelShape2[1:])
limit2 = numpy.sqrt(6. / (fanIn2 + fanOut2))
kernel2 = numpy.random.uniform(-limit, limit, kernelShape2)

padding2 = (1, 1)
stride2 = 1

if (padding2):
    padded2 = numpy.zeros((pool1.shape[0], pool1.shape[1] + 2*padding2[0], pool1.shape[2] + 2*padding2[1]))
    padded2[:, padding2[0]:pool1.shape[1]+padding2[0], padding2[1]:pool1.shape[2]+padding2[1]] = pool1
else:
    padded2 = pool1

conv2d2 = numpy.zeros((
    pool1.shape[0] * kernel2.shape[0],
    int(((pool1.shape[1] - kernel2.shape[1] + 2 * padding2[0]) / stride2) + 1),
    int(((pool1.shape[2] - kernel2.shape[2] + 2 * padding2[1]) / stride2) + 1)
))

for i in range(pool1.shape[0]):
    for x in range(conv2d2.shape[1]):
        for y in range(conv2d2.shape[2]):
            mul2 = padded2[i, x:x+kernel2.shape[1], y:y+kernel2.shape[2]] * kernel2
            conv2d2[i*kernel2.shape[0]:(i+1)*kernel2.shape[0], x, y] = numpy.sum(mul2, axis=(1, 2))

In [36]:
act2 = numpy.maximum(0, conv2d2)

In [37]:
pool2shape = (2, 2)

pool2 = numpy.zeros((act2.shape[0], int(act2.shape[1] / pool2shape[0]), int(act2.shape[2] / pool2shape[1])))

for x in range(pool2.shape[1]):
    for y in range(pool2.shape[2]):
        pool2[:, x, y] = numpy.max(act2[:,x*pool2shape[0]:(x+1)*pool2shape[0],y*pool2shape[1]:(y+1)*pool2shape[1]], axis=(1, 2))

### Flatten

In [38]:
flatten = pool2.reshape(pool2.shape[0],pool2.shape[1]*pool2.shape[2])
print(flatten.shape)

flattenTest = flatten.reshape(int(flatten.shape[0] / (32*64)), pool2.shape[1]*pool2.shape[2]*(32*64))
print(flattenTest.shape)

(65536, 49)
(32, 100352)


In [39]:
w, b = Xavier(flattenTest.shape[1], 10)

dot = flattenTest @ w
som = dot + b
act = 1 / (numpy.exp(-som) + 1)

print(act.shape)

(32, 10)


In [47]:
de_dact = (2 * (act - Y_batch)) / act.shape[0]

dact_dsum = numpy.exp(-som) / ((numpy.exp(-som) + 1) ** 2)
de_dsum = de_dact * dact_dsum

dsum_db = numpy.ones(b.shape)
de_db = de_dsum * dsum_db

dsum_ddot = numpy.ones(dot.shape)
de_ddot = de_dsum * dsum_ddot

ddot_dw = flattenTest
de_dw = de_ddot.T @ ddot_dw

In [48]:
ddot_dflattenTest = w

print(de_ddot.shape)
print(ddot_dflattenTest.shape)

de_dflattenTest = de_ddot @ ddot_dflattenTest.T
de_dflattenTest = de_dflattenTest.reshape(flatten.shape)

# skipping max pooling layer

de_dact2 = de_dflattenTest.reshape(pool2.shape)
dact2_dconv2d2 = 1 * (conv2d2 > 0)
de_dconv2d2 = de_dact2 * dact2_dconv2d2



(32, 10)
(100352, 10)


ValueError: operands could not be broadcast together with shapes (65536,7,7) (65536,14,14) 

In [46]:

b -= (numpy.sum(de_db) * ALPHA)
w -= (de_dw.T * ALPHA)